In [2]:
import pandas as pd
import numpy as np

In [3]:
# 컬럼명 변경
money_df = pd.DataFrame(columns=["ID", 'Money'])

for _ in range(15):
    money_df.loc[len(money_df)] = {
        'ID' : np.random.randint(1, 9),
        'Money' : np.random.randint(1,21) * 1000
    }
money_df

,ID,Money
0,3,10000
1,4,13000
2,5,3000
3,3,17000
4,3,12000
5,6,13000
6,5,17000
7,5,9000
8,2,6000
9,1,13000


In [17]:
money_df.rename(columns = {'ID' : 'UserID'}, inplace = True)
money_df

,UserID,Money
0,8,5000
1,2,18000
2,7,10000
3,2,3000
4,7,10000
5,4,16000
6,5,6000
7,1,12000
8,3,13000
9,3,8000


In [16]:
# groupby : sum, size, min .. 함수 : Series
money_df.groupby('UserID').sum()['Money'].reset_index()

,UserID,Money
0,1,12000
1,2,21000
2,3,26000
3,4,16000
4,5,18000
5,6,13000
6,7,20000
7,8,28000


In [20]:
# groupby : agg('sum'), agg('mean') .. : DataFrame
result_df = money_df.groupby('UserID').agg('sum')['Money'].reset_index()
result_df

,UserID,Money
0,1,12000
1,2,21000
2,3,26000
3,4,16000
4,5,18000
5,6,13000
6,7,20000
7,8,28000


In [ ]:
# fillna : NaN 을 특정 데이터로 채워줌

In [22]:
# sort_values : 정렬
result_df.sort_values('Money', ascending=False)

,UserID,Money
7,8,28000
2,3,26000
1,2,21000
6,7,20000
4,5,18000
3,4,16000
5,6,13000
0,1,12000


In [25]:
# 상위 3등까지 Money 평균
np.average(result_df.sort_values('Money', ascending=False)[:3]['Money'])

25000.0

### Pandas Pivot
- 데이처 프레임의 컬럼 데이터에서 index, column, value를 선택해서 데이터 프레임을 만드는 방법
- df.pivot(index, columns, values)
    - groupby 하고 pivot을 실행
- df.pivot_table(values, index, columns, aggfunc)

### pandas io
- 데이터 프레임을 저장, 로드

In [26]:
# load
titanic = pd.read_csv('train.csv')
titanic.tail(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [27]:
# save
titanic.to_csv('test.csv')

In [28]:
# 1. 성별, 좌석등급에 따른 데이터의 수
df1 = titanic.groupby(['Sex', 'Pclass']).size().reset_index(name = 'count')
df1

,Sex,Pclass,count
0,female,1,94
1,female,2,76
2,female,3,144
3,male,1,122
4,male,2,108
5,male,3,347


In [33]:
# pivot
result = df1.pivot('Sex', 'Pclass', 'count')
result

Pclass,1,2,3
Sex,,,
female,94,76,144
male,122,108,347


In [35]:
#pivot_table 이용
titanic['counts'] = 1
titanic

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,counts
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,1
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,1
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,1


In [39]:
result = titanic.pivot_table('counts', ['Pclass'], ['Survived'], aggfunc = np.sum)
result

Survived,0,1
Pclass,,
1,80,136
2,97,87
3,372,119


In [43]:
# total 추가
result['total'] = result[0] + result [1]
result

Survived,0,1,total
Pclass,,,
1,80,136,216
2,97,87,184
3,372,119,491


In [46]:
result.loc['total'] = result.loc[1] + result.loc[2] + result.loc[3]
result

Survived,0,1,total
Pclass,,,
1,80,136,216
2,97,87,184
3,372,119,491
total,549,342,891


### 선형회귀 분석
- 프리미어리그 데이터(득점, 실정, 승점)
- 득점, 실점 -> 승점 예측 하는 모델
- scikit-learn 패키지
    - 데이터 마이닝 및 데이터 분석, 모델을 위한 도구
    - 상업적으로 사용이 가능한 오픈소스

In [49]:
import pickle
from sklearn import linear_model # 선형회귀 모델
from sklearn.model_selection import train_test_split # 학습 데이터와 테스트 데이터를 나눠주는 모듈
from sklearn.metrics import mean_absolute_error # 모델을 평가해주는 모듈

### 분석 절차
- 데이터 로드
- 데이터 전처리 
    - 독립변수와 종속변수를 나눠줌
    - 학습 데이터와 테스트 데이터를 나눠줌
- 데이터 분석 : 선형회귀 모델
- 성능평가 : MAE
- 예측 코드 작성

In [52]:
# 1. 데이터 로드
p_df = pd.read_csv('premierleague.csv')
p_df.head()

,name,gf,ga,points
0,Manchester City,106,27,100
1,Manchester United,68,28,81
2,Tottenham Hotspur,74,36,77
3,Liverpool,84,38,75
4,Chelsea,62,38,70


In [54]:
# 2. 데이터 전처리 1
#독립변수, 종속변수 나누기
df_x = p_df[['gf','ga']]
df_y = p_df['points']

In [66]:
# 2. 데이터 전처리2
# 학습 데이터와 테스트 데이터로 나누기
train_x, test_x, train_y, test_y = train_test_split(df_x, df_y, test_size = 0.3, random_state = 1)

In [67]:
# 3. 데이터 분석 : 선형 회귀 모델
model = linear_model.LinearRegression()

In [68]:
model.fit(train_x, train_y)

LinearRegression()

In [69]:
# 4. 성능 평가 : MAE
pred_y = model.predict(test_x)
pred_y

array([80.88065736, 40.79778662, 50.9467527 , 46.31419247, 76.04209484,
       40.23228639])

In [77]:
test_y.values

array([75, 36, 54, 44, 77, 40])

In [83]:
pred_y = np.around(pred_y.flatten()).astype('int')
pred_y

array([81, 41, 51, 46, 76, 40])

In [85]:
mae = mean_absolute_error(test_y, pred_y)
round(mae, 2)

2.83

In [86]:
# 5. 예측 함수
def make_df(gf, ga):
    return pd.DataFrame({'gf' : [gf], 'ga' : [ga]})

In [91]:
gf, ga = 78, 30
result = int(model.predict(make_df(gf, ga)).flatten()[0])
result

82

In [93]:
# pickle 파일로 모델 저장하기
with open('p_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [94]:
with open('p_model.pkl', 'rb') as f:
    load_model = pickle.load(f)

In [96]:
gf, ga = 80, 30
result = int(load_model.predict(make_df(gf, ga)).flatten()[0])
result

83